# Advanced Delta Lake Features

Este notebook aborda recursos exclusivos do Delta Lake para otimização, gerenciamento de histórico e limpeza de dados.

## Objetivos de Aprendizado

Ao final desta lição, você será capaz de: 

* Usar `OPTIMIZE` para compactar arquivos pequenos 


* Usar `ZORDER` para indexar tabelas 


* Descrever a estrutura de diretórios dos arquivos Delta Lake 


* Revisar o histórico de transações da tabela 


* Consultar e restaurar versões anteriores da tabela 


* Limpar arquivos de dados obsoletos com `VACUUM` 



---

## 1. Criando uma Tabela Delta com Histórico

O código abaixo condensa diversas transações para popular nossa tabela de exemplo. 

```sql
CREATE TABLE students (id INT, name STRING, value DOUBLE);

INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

INSERT INTO students VALUES 
(4, "Ted", 4.7),
(5, "Tiffany", 5.5),
(6, "Vint", 5.3);

UPDATE students SET value = value + 1 WHERE name LIKE "T%";

DELETE FROM students WHERE value > 6;

```

In [0]:
%sql
DROP TABLE students;

CREATE TABLE students (id INT, name STRING, value DOUBLE);

INSERT INTO students VALUES (1, 'Alefe', 1.0);
INSERT INTO students VALUES (2, 'Natália', 2.5);
INSERT INTO students VALUES (3, 'João', 3.3);

INSERT INTO students VALUES
(4, 'Batman', 4.7),
(5, 'Flash', 5.5),
(6, 'Mulher Maravilha', 5.3);

UPDATE students SET value = value + 1 WHERE name LIKE 'B%';

DELETE FROM students WHERE value > 6;

In [0]:
%sql
SELECT * FROM students;

### Operação de MERGE

Criamos uma view temporária para realizar um `MERGE`, simulando inserções, atualizações e deleções simultâneas. 

```sql
CREATE OR REPLACE TEMP VIEW updates (id, name, value, type) AS VALUES
(2, "Omar", 15.2, "update"),
(3, "", null, "delete"),
(7, "Blue", 7.7, "insert"),
(11, "Diya", 8.8, "update");

MERGE INTO students b
USING updates u
ON b.id = u.id
WHEN MATCHED AND u.type = "update" THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete" THEN DELETE
WHEN NOT MATCHED AND u.type = "insert" THEN INSERT *;

```

---

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW updates (id, name, value, type) AS VALUES
(2, 'Omar', 15.2, 'update'),
(3, '', null, 'delete'),
(7, 'Blue', 7.7, 'insert'),
(11, 'Diya', 8.8, 'update');

MERGE INTO students b
USING updates u
ON b.id = u.id
WHEN MATCHED AND u.type = 'update' THEN UPDATE SET *
WHEN MATCHED AND u.type = 'delete' THEN DELETE 
WHEN NOT MATCHED AND u.type = 'insert' THEN INSERT *;


In [0]:
%sql
SELECT * FROM students
ORDER BY value DESC;

## 2. Examinando Detalhes da Tabela

O Delta Lake armazena metadados no Hive Metastore, mas os dados reais residem em armazenamento de objetos em nuvem. 

```sql
-- Ver metadados estendidos
DESCRIBE EXTENDED students;

-- Ver detalhes específicos como localização e número de arquivos
DESCRIBE DETAIL students;

```

### Exploração de Arquivos (Sistema de Arquivos)

Uma tabela Delta é composta por arquivos Parquet e um log de transações. 

```python
# Listar arquivos no diretório da tabela
display(dbutils.fs.ls(f"{DA.paths.user_db}/students"))

# Espiar o log de transações (_delta_log)
display(dbutils.fs.ls(f"{DA.paths.user_db}/students/_delta_log"))

```

> **Nota:** Cada transação gera um novo arquivo JSON no diretório `_delta_log`. O Delta Lake é indexado em 0. 
> 
> 

---

In [0]:
%sql
DESCRIBE EXTENDED students;

In [0]:
%sql
DESCRIBE DETAIL students;

## 3. Compactação e Indexação (OPTIMIZE)

Arquivos pequenos podem prejudicar a performance. O comando `OPTIMIZE` os compacta em arquivos maiores e mais eficientes. 

```sql
OPTIMIZE students
ZORDER BY id;

```

* 
**ZORDER:** Coloca dados com valores semelhantes nos mesmos arquivos, acelerando a recuperação de dados ao filtrar pelos campos especificados. 



---

In [0]:
%sql
OPTIMIZE students
ZORDER BY val

## 4. Histórico e Time Travel (Viagem no Tempo)

Como todas as mudanças são registradas no log, podemos revisar o histórico e consultar versões passadas. 

```sql
-- Revisar histórico de transações
DESCRIBE HISTORY students;

-- Consultar uma versão específica (Ex: Versão 3)
SELECT * FROM students VERSION AS OF 3;

```

In [0]:
%sql
DESCRIBE HISTORY students;

In [0]:
%sql
SELECT * FROM students VERSION AS OF 5;

### Rollback (Restaurar Versão)

Se você cometer um erro, como deletar todos os dados acidentalmente, pode restaurar a tabela. 

```sql
-- Exemplo de erro: deletar tudo
DELETE FROM students;

-- Restaurar para o estado anterior (Ex: Versão 8)
RESTORE TABLE students TO VERSION AS OF 8;

```

---

In [0]:
%sql
DELETE FROM students;

In [0]:
%sql
SELECT * FROM students;

In [0]:
%sql
RESTORE TABLE students TO VERSION AS OF 8;

In [0]:
%sql
SELECT * FROM students;

## 5. Limpeza de Arquivos Obsoletos (VACUUM)

O `VACUUM` remove permanentemente arquivos que não são mais necessários para a versão atual (ou versões dentro do período de retenção). 

> 
> **Aviso:** Após rodar o `VACUUM`, você perde a capacidade de fazer "Time Travel" para versões anteriores ao período de retenção especificado. 
> 
> 

```sql
-- Configurações para permitir retenção curta (apenas para fins de demonstração)
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
SET spark.databricks.delta.vacuum.logging.enabled = true;
```



In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
SET spark.databricks.delta.vacuum.logging.enabled = true;

```sql
-- Executar Dry Run para ver o que seria deletado
VACUUM students RETAIN 0 HOURS DRY RUN;

-- Executar a limpeza real
VACUUM students RETAIN 0 HOURS;

```

---

## Limpeza Final

Para remover as tabelas e arquivos criados nesta aula: 

```python
DA.cleanup()

```

In [0]:
%sql
VACUUM students RETAIN 0 HOURS DRY RUN;

In [0]:
%sql
VACUUM students RETAIN 0 HOURS;